In [1]:
!pip install openai

In [ ]:
def generateCode(model): 
    

In [10]:
import time 
import jsonlines
import os
import openai
from openai import OpenAI

openai.api_key = os.environ.get('OPENAI_API_KEY',"")
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY',""), )

In [25]:
def getCodeFormat(code_input):
    # Make an OPENAI API Call to only return the function call and its inputs
    '''EXAMPLE:
    INPUT: class Pair(object): def __init__(self, a, b): self.a = a self.b = b def max_chain_length(arr, n): max = 0 mcl = [1 for i in range(n)] for i in range(1, n): for j in range(0, i): if (arr[i].a > arr[j].b and mcl[i] < mcl[j] + 1): mcl[i] = mcl[j] + 1 for i in range(n): if (max < mcl[i]): max = mcl[i] return max
    
    OUTPUT: Class Pair(object) def max_chain_length(arr, n)
    '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": "Return only the function definition and its inputs from the code given by the user. Example: INPUT: class Pair(object): def __init__(self, a, b): self.a = a self.b = b def max_chain_length(arr, n): max = 0 mcl = [1 for i in range(n)] for i in range(1, n): for j in range(0, i): if (arr[i].a > arr[j].b and mcl[i] < mcl[j] + 1): mcl[i] = mcl[j] + 1 for i in range(n): if (max < mcl[i]): max = mcl[i] return max OUTPUT: Class Pair(object) def max_chain_length(arr, n) "
            },
            {
                "role": "user",
                "content": code_input
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    responseText = response.choices[0].message.content

    # Remove the ```python and ``` from the response
    responseText = responseText.replace("```python", "")
    responseText = responseText.replace("```", "")

    return responseText

    

In [30]:
def generateCode(modelType): 
    # Models: "gpt-4-1106-preview", "gpt-3.5-turbo-1106"
    response = client.chat.completions.create(
    model=modelType,
    messages=[
        {
            "role": "system",
            "content": "You are an expert in writing Python code. You will generate a Python Script that will complete the task outlined by the user. You will not provide any explanations, and only return the Python Script without the explanation of the code. "
        },
        {
            "role": "user",
            "content": userPrompt
        }
    ],
    temperature=1,
    max_tokens=2047,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    responseText = response.choices[0].message.content

    # Remove the ```python and ``` from the response
    responseText = responseText.replace("```python", "")
    responseText = responseText.replace("```", "")


    return responseText



    

In [37]:
def checkCorrectness(output, obj):
    # Execute the code:
    exec(output)
    
    assertion_code = obj['test_list']

    success = True
    for assertion in assertion_code:
        try:
            exec(assertion)
            print("Code Passed Assertions")
        except:
            # Continue to the next example and print the error
            print("Code Failed Assertions")
            print(assertion)
            success = False

    return success
    

    

In [38]:
max_loop = 10
i = 0

# GPT4 Model
# Open JSONL File:
with jsonlines.open('mbpp.jsonl') as reader: 
    # Loop through each example
    for obj in reader:
        print("Running Example: " + str(i))
        prompt = obj['text']
        code = obj['code']
        #print(obj['code'])
        # Call the getCodeFormat function to get the function call and its inputs
        function_call = getCodeFormat(code)
        print("Function Call")
        print(function_call)
        userPrompt =  prompt + " The name of the function should be "  + function_call + "\n\n"


        responseText_gpt4 = generateCode("gpt-4-1106-preview")
        responseText_gpt3_5 = generateCode("gpt-3.5-turbo-1106")

        # Check the correctness of the code
        success_gpt4 = checkCorrectness(responseText_gpt4, obj)
        success_gpt3_5 = checkCorrectness(responseText_gpt3_5, obj)
        
    

        # Write to JSONL File
        obj["method2_gpt4_output"] = responseText_gpt4
        obj["method2_gpt3_5_output"] = responseText_gpt3_5
        obj["method2_gpt4_success"] = success_gpt4
        obj["method2_gpt3_5_success"] = success_gpt3_5

        with jsonlines.open('mbpp_method2.jsonl', mode='a') as writer:
            writer.write(obj)
        
        
        
        time.sleep(5)

        
        # terminate loop for testing
        i += 1
        if i == max_loop:
            break


Running Example: 0
Function Call
def min_cost(cost, m, n)


In [74]:
print("Success: " + str(success) + " out of " + str(i))

Success: 16 out of 10


In [76]:
print("Success: " + str(success) + " out of " + str(i))

Success: 21 out of 10


In [77]:
# See number of examples in the JSONL file

with jsonlines.open('mbpp.jsonl') as reader:
    i = 0
    for obj in reader:
        i += 1
print("Number of Examples: " + str(i))

Number of Examples: 974


In [33]:
import re

# Read the "code" mbpp.jsonl file
with open('mbpp.jsonl', 'r') as file:
    # Initialize a set to store the unique package names
    packages = set()

    # Loop through each line in the file
    for line in file:
        # Use regular expressions to find the package names
        matches = re.findall(r'import\s+(\w+)', line)
        
        # Add the package names to the set
        packages.update(matches)

# Install the packages
for package in packages:
    !pip install {package}


ERROR: Could not find a version that satisfies the requirement maxsize (from versions: none)
ERROR: No matching distribution found for maxsize
ERROR: Could not find a version that satisfies the requirement itemgetter (from versions: none)
ERROR: No matching distribution found for itemgetter
ERROR: Could not find a version that satisfies the requirement array (from versions: none)
ERROR: No matching distribution found for array
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
ERROR: Could not find a version that satisfies the requirement bisect (from versions: none)
ERROR: No matching distribution found for bisect
  Preparing metadata (setup.py) ... done
  Created wheel for eq: filename=eq-0.0.10-py3-none-any.whl size=31906 sha256=95e38dc31f26b636acc74176b0782dd2da148c3fcd26aed53a91f4e1d4ceefb5
  Stored in directory: /Users/henrybae/Library/Caches/pip/wheels/eb/86/0b/f87c419524c84b0ceaf